## Below cells's calculate FID between printed input music sheet images with corresponding model generated handwritten music sheet images using cleanfid module.

Please make sure to update the path of input and model generated images as required.

In [3]:
from cleanfid import fid

In [11]:
#Path for input and generated images of UNet model

input_image = '/home/ec22221/cycleGan_tf_conda/UNet_Model/Test_Results/input'
generated_image = '/home/ec22221/cycleGan_tf_conda/UNet_Model/Test_Results/generated/'

In [12]:
score = fid.compute_fid(input_image, generated_image)
print(f"FID Score for UNet model: {score:.3f}")

compute FID between two folders
Found 200 images in the folder /home/ec22221/cycleGan_tf_conda/UNet_Model/Test_Results/input


FID input : 100%|███████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.85it/s]


Found 200 images in the folder /home/ec22221/cycleGan_tf_conda/UNet_Model/Test_Results/generated/


FID  : 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  5.79it/s]


FID Score for UNet model: 69.830


In [9]:
#Path for input and generated images of ResNet model


input_image = '/home/ec22221/cycleGan_tf_conda/ResNet_Model/Test_Results/input/'
generated_image = '/home/ec22221/cycleGan_tf_conda/ResNet_Model/Test_Results/generated/'

In [10]:
score = fid.compute_fid(input_image, generated_image)
print(f"FID Score for ResNet model: {score:.3f}")

compute FID between two folders
Found 200 images in the folder /home/ec22221/cycleGan_tf_conda/ResNet_Model/Test_Results/input/


FID  : 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  5.19it/s]


Found 200 images in the folder /home/ec22221/cycleGan_tf_conda/ResNet_Model/Test_Results/generated/


FID  : 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  6.39it/s]


FID Score for ResNet model: 60.524


## Below cells's calculate FID between printed input music sheet images with corresponding model generated handwritten music sheet images using inceptionV3 module.

Please make sure to update the path of input and model generated images as required.

In [13]:
import os
import numpy as np
from scipy.linalg import sqrtm
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array


2023-08-23 12:44:54.718290: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-23 12:44:56.150076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 12:44:59.956050: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
# Function to calculate FID
def calculate_fid(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)

    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)

    # calculate sqrt of product between covariances
    covmean = sqrtm(sigma1.dot(sigma2))

    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


In [15]:
# Function to load all images in a directory
def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        # load image
        image = load_img(directory + filename, target_size=(299, 299))
        # convert image pixels to numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the Inception model
        image = preprocess_input(image)
        images.append(image)
    return np.vstack(images)

In [16]:
# Prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

2023-08-23 12:45:09.504582: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22054 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:3b:00.0, compute capability: 7.5
2023-08-23 12:45:09.505117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 15844 MB memory:  -> device: 1, name: Quadro RTX 6000, pci bus id: 0000:d9:00.0, compute capability: 7.5


In [20]:
inp = '/home/ec22221/cycleGan_tf_conda/UNet_Model/Test_Results/input/'
generated = '/home/ec22221/cycleGan_tf_conda/UNet_Model/Test_Results/generated/'

images1 = load_images(inp)
images2 = load_images(generated)

# Calculate FID
fid = calculate_fid(model, images1, images2)
print('FID for UNet Model:', fid)

7/7 [==============================] - 0s 43ms/step
FID for UNet Model: 109.63310978256399


In [21]:
inp = '/home/ec22221/cycleGan_tf_conda/ResNet_Model/Test_Results/input/'
generated = '/home/ec22221/cycleGan_tf_conda/ResNet_Model/Test_Results/generated/'

images1 = load_images(inp)
images2 = load_images(generated)

# Calculate FID
fid = calculate_fid(model, images1, images2)
print('FID for ResNet model:', fid)

7/7 [==============================] - 0s 42ms/step
FID for ResNet model: 95.88087217882787


## Below cells's Inception Score of model generated handwritten music sheet images.

Please make sure to update the path of input and model generated images as required.

In [22]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from scipy.stats import entropy


In [23]:
# Function to calculate Inception Score
def calculate_inception_score(images, num_splits=10):
    # Load InceptionV3 model
    model = InceptionV3(include_top=True, weights='imagenet')

    # Preprocess images
    images = preprocess_input(images)

    # Get predicted probabilities for each class
    preds = model.predict(images)

    # Calculate Inception Score
    scores = []
    n = len(preds) // num_splits
    for i in range(num_splits):
        part = preds[(i * n):((i + 1) * n), :]
        kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)

In [24]:
# Function to load all images in a directory
def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        # load image
        image = load_img(directory + filename, target_size=(299, 299))
        # convert image pixels to numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        images.append(image)
    return np.vstack(images)

In [27]:
# Load images from a folder
folder_path = '/home/ec22221/cycleGan_tf_conda/UNet_Model/Test_Results/generated/'  # Please replace this with the actual path to your folder
images = load_images(folder_path)

# Calculate Inception Score
mean, std = calculate_inception_score(images)
print('Inception Score of images generated using UNet model: Mean', mean, 'Std', std)

7/7 [==============================] - 1s 48ms/step
Inception Score of images generated using UNet model: Mean 1.843847 Std 0.1415977


In [26]:
# Load images from a folder
folder_path = '/home/ec22221/cycleGan_tf_conda/ResNet_Model/Test_Results/generated/'  # Please replace this with the actual path to your folder
images = load_images(folder_path)

# Calculate Inception Score
mean, std = calculate_inception_score(images)
print('Inception Score  of images generated using ResNet model: Mean', mean, 'Std', std)

7/7 [==============================] - 1s 47ms/step
Inception Score  of images generated using ResNet model: Mean 1.6582989 Std 0.1504261
